# Homework06

Exercises to practice pandas, data analysis and regression

## Goals

- Understand the effects of pre-processing data
- Get familiar with the ML flow: encode -> normalize -> train -> evaluate
- Understand the difference between regression and classification tasks
- Build intuition for different regression models

### Setup

Run the following 2 cells to import all necessary libraries and helpers for this homework.

In [ ]:
!wget -q https://github.com/PSAM-5020-2025S-A/5020-utils/raw/main/src/data_utils.py

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder

from data_utils import object_from_json_url
from data_utils import StandardScaler
from data_utils import LinearRegression, SGDRegressor
from data_utils import regression_error

### Load Dataset

Let's load up the full [ANSUR](https://www.openlab.psu.edu/ansur2/) dataset that we looked at briefly in [Week 02](https://github.com/DM-GY-9103-2024F-H/WK02).

This is the dataset that has anthropometric information about U.S. Army personnel.

#### WARNING

Like we mentioned in class, this dataset is being used for these exercises due to the level of detail in the dataset and the rigorous process that was used in collecting the data.

This is a very specific dataset and should not be used to draw general conclusions about people, bodies, or anything else that is not related to the distribution of physical features of U.S. Army personnel.

In [ ]:
# Load Dataset
ANSUR_FILE = "https://raw.githubusercontent.com/PSAM-5020-2025S-A/5020-utils/main/datasets/json/ansur.json"
ansur_data = object_from_json_url(ANSUR_FILE)

# Look at first 2 records
ansur_data[:2]

#### Nested data

This is that *nested* dataset from Week 02.

# 🤔

Let's load it into a `DataFrame` to see what happens.

In [ ]:
# Read into DataFrame
ansur_df = pd.DataFrame.from_records(ansur_data)
ansur_df.head()


# 😓🙄

That didn't work too well. We ended up with objects in our columns.

Luckily, our `DataFrame` library has a function called [`json_normalize()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html) that can help.

In [ ]:
# Read into DataFrame
ansur_df = pd.json_normalize(ansur_data)
ansur_df.head()

Much better. `DataFrames` are magic.

#### Data Exploration

Before we start creating models, let's do a little bit of data analysis and get a feeling for the shapes, distributions and relationships of our data.

1. Print `min`, `max` and `average` values for all of the features.
2. Print `covariance` tables for `age`, `ear.length` and `head.circumference`.
3. Plot `age`, `ear.length` and `head.circumference` versus the $1$ *feature* that is most correlated to each of them.

Don't forget to *encode* and *normalize* the data.

# Work on Data Exploration here

# Encode non-numerical features
ansur_df['female'] = (ansur_df['gender'] == 'F').astype(int)
ansur_df.drop(columns=['gender'], inplace=True)

# 1. Print min, max, avg
# ansur_df.describe()
print("Min:", ansur_df.min())
print("Max:", ansur_df.max())
print("Av:", ansur_df.mean())

In [ ]:
ansur_df['female'] = (ansur_df['gender'] == 'F').astype(int)
ansur_df.drop(columns=['gender'], inplace=True)
print("Min:", ansur_df.min())
print("Max:", ansur_df.max())
print("Av:", ansur_df.mean())

In [ ]:

### Normalize all data

ansur_scaler = StandardScaler()
ansur_scaled = ansur_scaler.fit_transform(ansur_df)

## 2. Print Covariances
print(ansur_df.cov())

## 3. Plot features most correlated to age, ear length and head circumference
correlations_ansur=ansur_df.corr()
print(correlations_ansur)


#plotting the important ones now
#age:
plt.scatter( ansur_df["ear.length"], ansur_df.age, marker='o', linestyle='', alpha=0.3)
plt.xlabel("ear length ")
plt.ylabel("age")
plt.show()

#ear length
plt.scatter(ansur_df.weight, ansur_df["ear.length"], marker='o', linestyle='', alpha=0.3)
plt.xlabel("Weight")
plt.ylabel("Ear Length")
plt.show()


#head circumference
plt.scatter(ansur_df["head.height"],ansur_df["head.circumference"] ,  marker='o', linestyle='', alpha=0.3)
plt.xlabel("head height")
plt.ylabel("head circumference")
plt.show()

### Interpretation

<span style="color:hotpink;">
Does anything stand out about these graphs? Or the correlations?<br>
Are correlations symmetric? Does the feature most correlated to ear length also have ear length as its most correlated pair?
</span>

<span style="color:hotpink;">Ear Length and weight definitely have a positive correlation to them, and head circumference and height definitely have a positive correlation.
Age and ear length is not super correlated in the most direct way, which makes sense as well.
Head height being correlated to circumference sounds like a sort of a circularity bias.</span>

### Regression

Now, we want to create a regression model to predict `head.circumference` from the data.

From our [Week 06](https://github.com/PSAM-5020-2025S-A/WK06) notebook, we can create a regression model by following these steps:

1. Load dataset (done! 🎉)
2. Encode label features as numbers (done! ⚡️)
3. Normalize the data (done! 🍾)
4. Separate the outcome variable and the input features
5. Create a regression model using all features
6. Run model on training data and measure error
7. Plot predictions and interpret results
8. Run model on test data, measure error, plot predictions, interpret results

In [ ]:
# Work on Regression Model here

## Separate outcome variable and input features
#output
headCircumference = ansur_df["head.circumference"]

#input
features= ansur_df[["head.height", "weight", "span"]]

## Create a regression model
ansur_model= LinearRegression()
ansur_model.fit(features, headCircumference)

## Measure error on training data
prediction_scaled = ansur_model.predict(features)
# Un-normalize the data

predicted = ansur_scaler.inverse_transform(prediction_scaled)
regression_error(ansur_df["head.circumference"], predicted["head.circumference"])

## Plot predictions and interpret results

plt.plot(ansur_df["head.circumference"], predicted["head.circumference"], marker='', color='r')
plt.title("predicted vs original head circumference")
plt.xlabel("original")
plt.ylabel("predicted")
plt.show()


In [ ]:
## Load Test Data
ANSUR_TEST_FILE = "https://raw.githubusercontent.com/PSAM-5020-2025S-A/5020-utils/main/datasets/json/ansur-test.json"

ansur_test_data = object_from_json_url(ANSUR_TEST_FILE)
ansur_test_df = pd.json_normalize(ansur_test_data)

ansur_test_encoded_df = ansur_test_df.copy()

g_vals = ansur_encoder.transform(ansur_test_df[["gender"]].values)
ansur_test_encoded_df[["gender"]] = g_vals

ansur_test_scaled_df = ansur_scaler.transform(ansur_test_encoded_df)

In [ ]:
## Run model on test data

## Measure error on test data

## Plot predictions and interpret results

### Interpretation

<span style="color:hotpink;">
How well does your model perform?<br>
How could you improve it?<br>
Are there ranges of circumferences that don't get predicted well?
</span>

<span style="color:hotpink;">EDIT THIS CELL WITH ANSWER</span>